In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [25]:
school = 'University of California, Riverside' # change to your school's name

In [27]:
df = pd.read_csv(school + '.csv') 

# replace wrong locations(string 'connections' in them) to nulls
no_value = df.current_location.str.contains('connections') 
df.loc[no_value,'current_location'] = np.NAN

# remove some strings not identified by geocode (!you can add more depending on your data)
l = ['Metropolitan','Area', 'Township of', 'Region', 'Greater', 'Metroplex', 'District']
df['current_location'] = df['current_location'].str.replace('|'.join(l), '', regex=True)

geolocator = Nominatim(user_agent="course_project")

In [32]:
geolocator = Nominatim(user_agent="course_project")
location = geolocator.geocode('San Francisco, California, United States', timeout = 600, addressdetails=True)
location.raw['address']

{'city': 'San Francisco',
 'state': 'California',
 'country': 'United States',
 'country_code': 'us'}

In [34]:
# generate a dictionary mapping unique locations to corresponding coordinates (3 mins)
coor = {}
county = {}
city = {}
unique_locs = df.current_location.unique()
for loc in unique_locs:
    location = geolocator.geocode(loc, timeout = 600,addressdetails=True)
    if pd.isna(location) or pd.isna(loc):
        coor[loc] = np.NAN
        county[loc] = np.NAN
    else:
        coor[loc] = (location.latitude, location.longitude)

        print(location.raw['address'])
        try:
            county[loc] = location.raw['address']['county'].split(' ')[0]
        except:
            try:
                city[loc] = location.raw['address']['city']
            except:
                pass

{'city': 'Los Angeles', 'county': 'Los Angeles County', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
{'town': 'Newport Beach', 'city': 'Newport Beach', 'county': 'Orange County', 'state': 'California', 'postcode': '92658', 'country': 'United States', 'country_code': 'us'}
{'city': 'San Francisco', 'state': 'California', 'country': 'United States', 'country_code': 'us'}
{'town': 'Allen', 'county': 'Collin County', 'state': 'Texas', 'country': 'United States', 'country_code': 'us'}
{'town': 'Coppell', 'county': 'Dallas County', 'state': 'Texas', 'postcode': '75019', 'country': 'United States', 'country_code': 'us'}
{'town': 'El Segundo', 'county': 'Los Angeles County', 'state': 'California', 'postcode': '90245', 'country': 'United States', 'country_code': 'us'}
{'city': 'Olympia', 'county': 'Thurston County', 'state': 'Washington', 'country': 'United States', 'country_code': 'us'}
{'city': 'Los Angeles', 'county': 'Los Angeles County', 'state': 'California', '

In [35]:
df['coordinate'] = df.apply(lambda row: coor[row.current_location], axis =1)
df['county'] = df.apply(lambda row: county[row.current_location], axis =1)
df['city'] = df.apply(lambda row: city[row.current_location], axis =1)

KeyError: 'San Francisco, California, United States'

In [181]:
#check a specific location
location = geolocator.geocode('Frankfurt Rhine', timeout = 600)
location

Location(Frankfurter Straße, Urdenbach, Stadtbezirk 9, Düsseldorf, Nordrhein-Westfalen, 40593, Deutschland, (51.1549523, 6.8854454, 0.0))

In [175]:
#check NAN values and revise current locations to be searchable (change/add more strings to remove in l)
df[(df.coordinate.isna()) & (df.current_location.notna())]

,current_location,current_work,previous_work,latest_Education,majro,search_school,coordinate
337,"Greenville-Spartanburg-Anderson, South Carolina","President/Owner, Financial Advisor, CFP®, CRPC...","Vice President, Financial Advisor, CFP®, CRPC®...",University of South Carolina - The Moore Schoo...,Brazil/Portuguese,"University of California, Riverside",NaN
548,Frankfurt Rhine-Main,Head of Banking and Partner Strategy and Execu...,Senior Adviser at Holland FinTech,"University of California, Riverside",German Language and Literature,"University of California, Riverside",NaN
796,"Greenville-Spartanburg-Anderson, South Carolina","President/Owner, Financial Advisor, CFP®, CRPC...","Vice President, Financial Advisor, CFP®, CRPC®...",University of South Carolina - The Moore Schoo...,Brazil/Portuguese,"University of California, Riverside",NaN
1001,Ezana is a startup entrepreneur currently prom...,CEO at Afri Kash,Service Management Officer at United Nations,"University of California, Riverside",Political Science,"University of California, Riverside",NaN
1013,Laila Mickelwait is the Founder and CEO of the...,Founder/CEO at Justice Defense Fund,Founder at New Reality International,University of Southern California,Public Diplomacy,"University of California, Riverside",NaN
1782,CEO at Cyphen Limited,CEO at Cyphen Limited,NaN,Peking University,国际经济,"University of California, Riverside",NaN
